In [1]:
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive
from simple_ascii_art import img2ascii_arr, img2ascii_arr_v2
from img_processing import preprocess_img

In [2]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")

print(orig_img.size)

(512, 512)


In [3]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq):
    proc_img = preprocess_img(orig_img, scale_factor, contrast, brightness, eq, quantize_colors)

    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(15, 5))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            quantize_colors=(0, 255, 1),
            eq=(0, 1, 0.01))

display(interactive_preprocess)

interactive(children=(FloatSlider(value=0.5, description='scale_factor', max=1.0, min=0.01, step=0.01), FloatS…

In [4]:
proc_img = interactive_preprocess.result
proc_img_arr = np.array(proc_img)

In [5]:
ascii_arr = img2ascii_arr(proc_img_arr)

In [6]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j],ascii_arr[i][j], sep='', end='')
    print('\n', end='')

                                                ....................................................::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::....................................................                                              
                                            ......................................................::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::......................................................                 

In [7]:
ascii_arr = img2ascii_arr_v2(proc_img_arr)

In [8]:
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

                                .:::::::::::::::::::::::::::::::::::::~~+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++~~:::::::::::::::::::::::::::::::::::::.                               
                             .:::::::::::::::::::::::::::::::::::~~+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++~~:::::::::::::::::::::::::::::::::::.                            
                          ..::::::::::::::::::::::::::::::::::~+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++~::::::::::::::::::::::::::::::::::..                         
                        .:::::::::::::::::::::::::::::::::~~+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++~~::::::::::::::::::::::::::::::